# Донавчання детектора об'єктів на датасеті ArTaxOr

Цей notebook містить повний пайплайн для донавчання моделі YOLOv8 на датасеті [ArTaxOr (Arthropod Taxonomy Orders Object Detection Dataset)](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset) з Kaggle.


## 1. Встановлення залежностей


In [ ]:
# Встановлення необхідних бібліотек
%pip install ultralytics>=8.0.0 -q
%pip install opencv-python>=4.8.0 -q
%pip install pillow>=10.0.0 -q
%pip install pyyaml>=6.0 -q
%pip install tqdm>=4.65.0 -q
%pip install kagglehub -q


## 2. Імпорт бібліотек


In [ ]:
import os
import json
import shutil
import random
from pathlib import Path
from PIL import Image
import xml.etree.ElementTree as ET
from tqdm import tqdm
from ultralytics import YOLO
import time
import kagglehub


## 3. Завантаження датасету з Kaggle


In [ ]:
# Налаштування шляхів
OUTPUT_DIR = "/kaggle/working"
DATASET_DIR = os.path.join(OUTPUT_DIR, "dataset")
YOLO_DATASET_DIR = os.path.join(OUTPUT_DIR, "yolo_dataset")

print(f"Робоча директорія: {OUTPUT_DIR}")

# Завантаження датасету через kagglehub
print("\nЗавантаження датасету через kagglehub..")
print("Це може зайняти деякий час при першому запуску...")

try:
    # Завантажуємо датасет через kagglehub
    dataset_path = kagglehub.dataset_download("mistag/arthropod-taxonomy-orders-object-detection-dataset")
    print(f"\nДатасет завантажено в: {dataset_path}")
    
    # Копіюємо датасет у робочу директорію для зручності
    if os.path.exists(dataset_path):
        # Шукаємо директорію ArTaxOr або використовуємо корінь датасету
        potential_artaxor = os.path.join(dataset_path, "ArTaxOr")
        if os.path.exists(potential_artaxor):
            DATASET_DIR = potential_artaxor
        else:
            # Шукаємо рекурсивно директорію з XML або JSON файлами
            for root, dirs, files in os.walk(dataset_path):
                xml_files = [f for f in files if f.endswith('.xml')]
                json_files = [f for f in files if f.endswith('.json')]
                if xml_files or json_files:
                    DATASET_DIR = root
                    break
            # Якщо не знайшли, використовуємо корінь
            if DATASET_DIR == os.path.join(OUTPUT_DIR, "dataset"):
                DATASET_DIR = dataset_path
        
        print(f"Використовується директорія датасету: {DATASET_DIR}")
        
        # Показуємо структуру датасету (перші 3 рівні)
        print("\nСтруктура датасету:")
        for root, dirs, files in os.walk(DATASET_DIR):
            level = root.replace(DATASET_DIR, '').count(os.sep)
            if level > 2:  # Обмежуємо глибину для читабельності
                continue
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            # Показуємо файли анотацій
            annotation_files = [f for f in files if f.endswith(('.xml', '.json'))]
            if annotation_files:
                for file in annotation_files[:3]:
                    print(f"{subindent}{file}")
                if len(annotation_files) > 3:
                    print(f"{subindent}... та ще {len(annotation_files) - 3} файлів анотацій")
            # Показуємо зображення
            image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            if image_files and not annotation_files:
                for file in image_files[:3]:
                    print(f"{subindent}{file}")
                if len(image_files) > 3:
                    print(f"{subindent}... та ще {len(image_files) - 3} зображень")
    
except Exception as e:
    print(f"\nПОМИЛКА при завантаженні датасету: {e}")
    print("\nСпробуйте альтернативний спосіб:")
    print("1. Додайте датасет через Add Data -> Search Datasets")
    print("2. Або переконайтеся, що у вас є доступ до датасету")
    DATASET_DIR = None


## 4. Функції для конвертації датасету у формат YOLO


In [ ]:
def parse_pascal_voc(xml_path):
    """
    Парсить XML файл у форматі Pascal VOC
    Повертає список bbox у форматі [class_id, x_center, y_center, width, height] (нормалізовані)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Отримуємо розміри зображення
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    boxes = []
    for obj in root.findall('object'):
        # Отримуємо клас (може бути name або class)
        class_name = obj.find('name').text
        
        # Отримуємо bbox
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        # Конвертуємо в YOLO формат (нормалізовані координати центру та розміри)
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        boxes.append({
            'class': class_name,
            'bbox': [x_center, y_center, width, height]
        })
    
    return boxes, img_width, img_height

def parse_azure_custom_vision(json_path, img_path=None, img_width=None, img_height=None):
    """
    Парсить JSON файл у форматі Azure Custom Vision або VoTT
    Повертає список bbox у форматі [class_id, x_center, y_center, width, height] (нормалізовані)
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    boxes = []
    
    # Отримуємо розміри зображення
    if img_width is None or img_height is None:
        if 'asset' in data:
            # Azure Custom Vision формат
            asset = data.get('asset', {})
            img_width = asset.get('width')
            img_height = asset.get('height')
        elif 'image' in data:
            # Альтернативний формат
            image_info = data.get('image', {})
            img_width = image_info.get('width')
            img_height = image_info.get('height')
    
    # Якщо розміри не знайдені в JSON, спробуємо завантажити зображення
    if (img_width is None or img_height is None) and img_path is not None:
        try:
            from PIL import Image
            with Image.open(img_path) as img:
                img_width, img_height = img.size
        except Exception as e:
            print(f"Попередження: не вдалося отримати розміри зображення {img_path}: {e}")
            return boxes, None, None
    
    if img_width is None or img_height is None:
        return boxes, None, None
    
    # Обробляємо регіони/анотації
    regions = []
    if 'regions' in data:
        regions = data['regions']
    elif 'annotations' in data:
        regions = data['annotations']
    elif 'objects' in data:
        regions = data['objects']
    
    for region in regions:
        # Отримуємо клас
        class_name = None
        if 'tags' in region and len(region['tags']) > 0:
            class_name = region['tags'][0]  # Беремо перший тег
        elif 'label' in region:
            class_name = region['label']
        elif 'name' in region:
            class_name = region['name']
        elif 'class' in region:
            class_name = region['class']
        
        if class_name is None:
            continue
        
        # Отримуємо bbox
        x_center = None
        y_center = None
        norm_width = None
        norm_height = None
        
        if 'boundingBox' in region:
            # Azure Custom Vision формат: {left, top, width, height}
            bbox_info = region['boundingBox']
            left = float(bbox_info.get('left', 0))
            top = float(bbox_info.get('top', 0))
            width = float(bbox_info.get('width', 0))
            height = float(bbox_info.get('height', 0))
            
            # Конвертуємо в YOLO формат
            x_center = (left + width / 2) / img_width
            y_center = (top + height / 2) / img_height
            norm_width = width / img_width
            norm_height = height / img_height
        elif 'bbox' in region:
            # Альтернативний формат: [x_min, y_min, width, height] або [x_min, y_min, x_max, y_max]
            bbox_coords = region['bbox']
            if len(bbox_coords) == 4:
                if bbox_coords[2] < 1 and bbox_coords[3] < 1:
                    # Вже нормалізовані координати
                    x_center, y_center, norm_width, norm_height = bbox_coords
                else:
                    # Абсолютні координати
                    x_min, y_min, x_max, y_max = bbox_coords
                    x_center = ((x_min + x_max) / 2) / img_width
                    y_center = ((y_min + y_max) / 2) / img_height
                    norm_width = (x_max - x_min) / img_width
                    norm_height = (y_max - y_min) / img_height
        elif 'points' in region:
            # Формат з точками
            points = region['points']
            if len(points) >= 2:
                x_coords = [p.get('x', p[0] if isinstance(p, (list, tuple)) else 0) for p in points]
                y_coords = [p.get('y', p[1] if isinstance(p, (list, tuple)) else 0) for p in points]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                x_center = ((x_min + x_max) / 2) / img_width
                y_center = ((y_min + y_max) / 2) / img_height
                norm_width = (x_max - x_min) / img_width
                norm_height = (y_max - y_min) / img_height
        
        if x_center is not None and y_center is not None:
            boxes.append({
                'class': class_name,
                'bbox': [x_center, y_center, norm_width, norm_height]
            })
    
    return boxes, img_width, img_height

def parse_coco(json_path):
    """
    Парсить JSON файл у форматі COCO
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Створюємо мапи для швидкого пошуку
    images = {img['id']: img for img in data['images']}
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Групуємо анотації по зображенням
    annotations_by_image = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(ann)
    
    return images, categories, annotations_by_image

def convert_coco_to_yolo(images, categories, annotations_by_image, class_mapping):
    """
    Конвертує анотації COCO у формат YOLO
    """
    result = {}
    for image_id, image_info in images.items():
        img_width = image_info['width']
        img_height = image_info['height']
        file_name = image_info['file_name']
        
        boxes = []
        if image_id in annotations_by_image:
            for ann in annotations_by_image[image_id]:
                category_id = ann['category_id']
                class_name = categories[category_id]
                
                if class_name not in class_mapping:
                    continue
                
                class_id = class_mapping[class_name]
                
                # COCO bbox формат: [x_min, y_min, width, height]
                x_min, y_min, width, height = ann['bbox']
                
                # Конвертуємо в YOLO формат
                x_center = (x_min + width / 2) / img_width
                y_center = (y_min + height / 2) / img_height
                norm_width = width / img_width
                norm_height = height / img_height
                
                boxes.append({
                    'class_id': class_id,
                    'bbox': [x_center, y_center, norm_width, norm_height]
                })
        
        result[file_name] = boxes
    
    return result

def detect_format(dataset_path):
    """
    Визначає формат анотацій датасету
    """
    dataset_path = Path(dataset_path)
    
    if not dataset_path.exists():
        print(f"ПОМИЛКА: Директорія не існує: {dataset_path}")
        return None, None
    
    print(f"\nПошук анотацій в: {dataset_path}")
    
    # Перевірка на Pascal VOC (XML файли)
    xml_files = list(dataset_path.rglob("*.xml"))
    if xml_files:
        print(f"Знайдено {len(xml_files)} XML файлів (Pascal VOC формат)")
        print(f"Приклад файлів:")
        for xml_file in xml_files[:3]:
            print(f"  - {xml_file}")
        if len(xml_files) > 3:
            print(f"  ... та ще {len(xml_files) - 3} файлів")
        return "pascal_voc", xml_files
    
    # Перевірка на JSON файли
    json_files = list(dataset_path.rglob("*.json"))
    if json_files:
        print(f"Знайдено {len(json_files)} JSON файлів")
        
        # Спочатку перевіряємо, чи є JSON файли в папках annotations
        annotation_json_files = []
        for json_file in json_files:
            if 'annotations' in str(json_file) or json_file.parent.name == 'annotations':
                annotation_json_files.append(json_file)
        
        # Якщо знайшли JSON файли в annotations папках, перевіряємо їх формат
        if annotation_json_files:
            print(f"Знайдено {len(annotation_json_files)} JSON файлів в папках annotations")
            # Перевіряємо структуру перших кількох файлів
            azure_format_count = 0
            coco_format_count = 0
            
            for json_file in annotation_json_files[:5]:  # Перевіряємо перші 5 файлів
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        
                        # Перевіряємо структуру COCO
                        has_images = 'images' in data
                        has_annotations = 'annotations' in data
                        has_categories = 'categories' in data
                        
                        if has_images and has_annotations and has_categories:
                            coco_format_count += 1
                            continue
                        
                        # Перевіряємо структуру Azure Custom Vision/VoTT
                        has_asset = 'asset' in data
                        has_regions = 'regions' in data
                        has_annotations_alt = 'annotations' in data and not has_images  # annotations без images = не COCO
                        has_objects = 'objects' in data
                        
                        # Azure Custom Vision має asset + regions, або regions, або annotations (як список об'єктів)
                        if (has_asset and has_regions) or (has_regions) or (has_annotations_alt and isinstance(data.get('annotations'), list)):
                            azure_format_count += 1
                except Exception as e:
                    continue
            
            # Визначаємо формат на основі перевірки
            if azure_format_count > 0 and coco_format_count == 0:
                print(f"Виявлено Azure Custom Vision/VoTT формат ({azure_format_count} з {min(5, len(annotation_json_files))} перевірених файлів)")
                return "azure_custom_vision", annotation_json_files
            elif coco_format_count > 0:
                # Шукаємо головний COCO файл (не в annotations)
                for json_file in json_files:
                    if 'annotations' not in str(json_file) or json_file.parent.name != 'annotations':
                        try:
                            with open(json_file, 'r', encoding='utf-8') as f:
                                data = json.load(f)
                                if 'images' in data and 'annotations' in data and 'categories' in data:
                                    print(f"Знайдено COCO формат в: {json_file}")
                                    return "coco", json_file
                        except:
                            continue
        
        # Якщо не знайшли в annotations, перевіряємо всі JSON файли на COCO
        for json_file in json_files[:20]:  # Перевіряємо більше файлів
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # Перевіряємо структуру COCO
                    has_images = 'images' in data
                    has_annotations = 'annotations' in data
                    has_categories = 'categories' in data
                    
                    if has_images and has_annotations and has_categories:
                        print(f"Знайдено COCO формат в: {json_file}")
                        return "coco", json_file
            except json.JSONDecodeError as e:
                continue
            except Exception as e:
                continue
    
    # Діагностика: показуємо що знайдено
    print("\nДіагностика:")
    print(f"Перевірено директорію: {dataset_path}")
    
    # Показуємо всі файли в директорії
    all_files = list(dataset_path.rglob("*"))
    files_by_ext = {}
    for file in all_files:
        if file.is_file():
            ext = file.suffix.lower()
            if ext not in files_by_ext:
                files_by_ext[ext] = []
            files_by_ext[ext].append(file)
    
    print(f"\nЗнайдені файли за розширенням:")
    for ext, files in sorted(files_by_ext.items()):
        print(f"  {ext}: {len(files)} файлів")
        if len(files) <= 5:
            for f in files:
                print(f"    - {f}")
        else:
            for f in files[:3]:
                print(f"    - {f}")
            print(f"    ... та ще {len(files) - 3} файлів")
    
    # Перевіряємо чи є піддиректорії
    subdirs = [d for d in dataset_path.iterdir() if d.is_dir()]
    if subdirs:
        print(f"\nПіддиректорії:")
        for subdir in subdirs[:10]:
            print(f"  - {subdir.name}/")
        if len(subdirs) > 10:
            print(f"  ... та ще {len(subdirs) - 10} директорій")
    
    return None, None


In [ ]:
def convert_dataset(input_path, output_path, train_split=0.8, val_split=0.1):
    """
    Конвертує датасет у формат YOLO
    
    Args:
        input_path: Шлях до вхідного датасету
        output_path: Шлях для вихідного датасету YOLO
        train_split: Частка даних для навчання
        val_split: Частка даних для валідації
    """
    input_path = Path(input_path)
    output_path = Path(output_path)
    
    # Створюємо структуру директорій YOLO
    for split in ['train', 'val', 'test']:
        (output_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (output_path / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Перевіряємо чи існує вхідна директорія
    if not input_path.exists():
        print(f"ПОМИЛКА: Вхідна директорія не існує: {input_path}")
        print("Переконайтеся, що датасет завантажено правильно")
        return
    
    # Визначаємо формат датасету
    format_type, format_data = detect_format(input_path)
    
    if format_type is None:
        print("\n" + "="*60)
        print("ПОМИЛКА: не вдалося визначити формат анотацій")
        print("="*60)
        print("Підтримувані формати:")
        print("  - Pascal VOC: XML файли з анотаціями")
        print("  - COCO: JSON файли зі структурою {images, annotations, categories}")
        print("  - Azure Custom Vision/VoTT: JSON файли з {asset, regions} в папках annotations/")
        print("\nМожливі причини:")
        print("  1. Датасет має інший формат анотацій")
        print("  2. Анотації знаходяться в іншій директорії")
        print("  3. Датасет не містить файлів анотацій")
        print("\nПеревірте вивід діагностики вище для деталей")
        print("="*60)
        return
    
    print(f"Виявлено формат: {format_type}")
    
    # Збираємо всі класи
    class_names = set()
    
    if format_type == "pascal_voc":
        xml_files = format_data
        for xml_file in tqdm(xml_files, desc="Збір класів"):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                class_names.add(class_name)
        
        # Створюємо мапінг класів
        class_names = sorted(list(class_names))
        class_mapping = {name: idx for idx, name in enumerate(class_names)}
        
        # Конвертуємо файли
        all_files = []
        for xml_file in tqdm(xml_files, desc="Конвертація"):
            # Шукаємо відповідне зображення
            img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            img_file = None
            for ext in img_extensions:
                potential_img = xml_file.parent / (xml_file.stem + ext)
                if potential_img.exists():
                    img_file = potential_img
                    break
            
            if img_file is None:
                continue
            
            boxes, img_width, img_height = parse_pascal_voc(xml_file)
            
            # Записуємо YOLO формат
            yolo_labels = []
            for box in boxes:
                class_id = class_mapping[box['class']]
                bbox = box['bbox']
                yolo_labels.append(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
            
            all_files.append((img_file, yolo_labels))
    
    elif format_type == "coco":
        json_file = format_data
        images, categories, annotations_by_image = parse_coco(json_file)
        
        # Створюємо мапінг класів
        class_names = sorted(list(categories.values()))
        class_mapping = {name: idx for idx, name in enumerate(class_names)}
        
        # Конвертуємо
        yolo_data = convert_coco_to_yolo(images, categories, annotations_by_image, class_mapping)
        
        # Знаходимо зображення
        img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        all_files = []
        for file_name, boxes in tqdm(yolo_data.items(), desc="Конвертація"):
            img_file = None
            for ext in img_extensions:
                potential_img = input_path / file_name
                if potential_img.exists():
                    img_file = potential_img
                    break
                # Шукаємо рекурсивно
                for found_img in input_path.rglob(file_name):
                    img_file = found_img
                    break
            
            if img_file is None or not img_file.exists():
                continue
            
            yolo_labels = []
            for box in boxes:
                yolo_labels.append(
                    f"{box['class_id']} {box['bbox'][0]:.6f} {box['bbox'][1]:.6f} "
                    f"{box['bbox'][2]:.6f} {box['bbox'][3]:.6f}"
                )
            
            all_files.append((img_file, yolo_labels))
    
    elif format_type == "azure_custom_vision":
        json_files = format_data
        
        # Групуємо JSON файли по директоріям (кожна директорія класу має свою папку annotations)
        json_by_dir = {}
        for json_file in json_files:
            parent_dir = json_file.parent.parent  # Директорія класу
            if parent_dir not in json_by_dir:
                json_by_dir[parent_dir] = []
            json_by_dir[parent_dir].append(json_file)
        
        # Збираємо всі класи (використовуємо множину для зберігання)
        classes_set = set(class_names)  # Створюємо копію множини
        
        for json_file in tqdm(json_files, desc="Збір класів"):
            try:
                boxes, _, _ = parse_azure_custom_vision(json_file)
                for box in boxes:
                    if box.get('class'):
                        classes_set.add(box['class'])
            except Exception as e:
                continue
        
        # Створюємо мапінг класів (поки що без нових класів)
        class_names = sorted(list(classes_set))
        class_mapping = {name: idx for idx, name in enumerate(class_names)}
        
        # Конвертуємо файли
        all_files = []
        img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        
        # Обробляємо кожну директорію окремо
        for parent_dir, dir_json_files in json_by_dir.items():
            # Отримуємо всі зображення в цій директорії
            all_images = []
            for ext in img_extensions:
                all_images.extend(list(parent_dir.glob(f"*{ext}")))
                all_images.extend(list(parent_dir.glob(f"*{ext.upper()}")))
            
            # Сортуємо для узгодженості
            all_images = sorted(all_images, key=lambda x: x.name)
            dir_json_files = sorted(dir_json_files, key=lambda x: x.name)
            
            # Створюємо мапу між JSON та зображеннями
            # Якщо кількість збігається, зв'язуємо за порядком
            json_to_img = {}
            if len(dir_json_files) == len(all_images):
                # Точна відповідність - зв'язуємо за порядком
                for json_file, img_file in zip(dir_json_files, all_images):
                    json_to_img[json_file] = img_file
            else:
                # Намагаємося знайти відповідність за назвою
                for json_file in dir_json_files:
                    json_stem = json_file.stem
                    # Видаляємо суфікс "-asset" якщо є
                    if json_stem.endswith('-asset'):
                        json_stem = json_stem[:-6]
                    
                    img_file = None
                    # Шукаємо за точним ім'ям
                    for ext in img_extensions:
                        potential_img = parent_dir / (json_stem + ext)
                        if potential_img.exists():
                            img_file = potential_img
                            break
                    
                    # Якщо не знайшли, шукаємо за частиною назви
                    if img_file is None:
                        for img_path in all_images:
                            img_stem = img_path.stem
                            if json_stem in img_stem or img_stem in json_stem:
                                img_file = img_path
                                break
                    
                    if img_file is not None:
                        json_to_img[json_file] = img_file
        
        # Обробляємо всі JSON файли
        for json_file in tqdm(json_files, desc="Конвертація"):
            try:
                # Отримуємо відповідне зображення
                img_file = json_to_img.get(json_file)
                
                if img_file is None or not img_file.exists():
                    continue
                
                # Отримуємо назву директорії класу
                parent_dir = json_file.parent.parent
                dir_class = parent_dir.name if parent_dir else None
                
                # Парсимо анотації
                boxes, img_width, img_height = parse_azure_custom_vision(json_file, img_path=str(img_file))
                
                if boxes is None or len(boxes) == 0:
                    # Якщо анотацій немає, але зображення є, можливо клас визначається з назви директорії
                    # Назва директорії - це назва класу
                    if dir_class and dir_class not in ['annotations', 'images', 'labels']:
                        classes_set.add(dir_class)
                        # Створюємо один bbox на все зображення (якщо потрібно)
                        # Але краще пропустити файли без анотацій
                    continue
                
                # Якщо в анотаціях немає класу, але є назва директорії, використовуємо її
                for box in boxes:
                    if not box.get('class') or box['class'] == '':
                        if dir_class and dir_class not in ['annotations', 'images', 'labels']:
                            box['class'] = dir_class
                            classes_set.add(dir_class)
                
                # Записуємо YOLO формат
                yolo_labels = []
                for box in boxes:
                    class_name = box.get('class')
                    if not class_name:
                        continue
                    if class_name not in class_mapping:
                        # Оновлюємо мапінг, якщо знайшли новий клас
                        classes_set.add(class_name)
                        class_names = sorted(list(classes_set))
                        class_mapping = {name: idx for idx, name in enumerate(class_names)}
                    class_id = class_mapping[class_name]
                    bbox = box['bbox']
                    yolo_labels.append(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
                
                if len(yolo_labels) > 0:
                    all_files.append((img_file, yolo_labels))
            except Exception as e:
                print(f"Помилка при обробці {json_file}: {e}")
                continue
        
        # Оновлюємо class_names з classes_set перед створенням файлів
        if format_type == "azure_custom_vision":
            class_names = sorted(list(classes_set))
    
    # Розділяємо на train/val/test
    random.seed(42)
    random.shuffle(all_files)
    
    n_total = len(all_files)
    n_train = int(n_total * train_split)
    n_val = int(n_total * val_split)
    
    splits = {
        'train': all_files[:n_train],
        'val': all_files[n_train:n_train + n_val],
        'test': all_files[n_train + n_val:]
    }
    
    # Копіюємо файли та створюємо labels
    for split_name, files in splits.items():
        print(f"\nОбробка {split_name}: {len(files)} файлів")
        for img_file, yolo_labels in tqdm(files, desc=f"Копіювання {split_name}"):
            # Копіюємо зображення
            dest_img = output_path / split_name / 'images' / img_file.name
            shutil.copy2(img_file, dest_img)
            
            # Створюємо label файл
            label_file = output_path / split_name / 'labels' / (img_file.stem + '.txt')
            with open(label_file, 'w') as f:
                f.write('\n'.join(yolo_labels))
    
    # Створюємо файл з назвами класів
    classes_file = output_path / 'classes.txt'
    with open(classes_file, 'w') as f:
        f.write('\n'.join(class_names))
    
    # Створюємо data.yaml для YOLO
    yaml_content = f"""# ArTaxOr Dataset Configuration
path: {output_path.absolute()}
train: train/images
val: val/images
test: test/images

# Classes
nc: {len(class_names)}
names: {class_names}
"""
    
    yaml_file = output_path / 'data.yaml'
    with open(yaml_file, 'w') as f:
        f.write(yaml_content)
    
    print(f"\nКонвертацію завершено!")
    print(f"Вихідний датасет: {output_path}")
    print(f"Кількість класів: {len(class_names)}")
    print(f"Класи: {', '.join(class_names)}")
    print(f"Конфігурація: {yaml_file}")


In [ ]:
# Конвертуємо датасет
convert_dataset(
    input_path=DATASET_DIR,
    output_path=YOLO_DATASET_DIR,
    train_split=0.8,
    val_split=0.1
)


## 6. Перевірка структури конвертованого датасету


In [ ]:
# Перевіряємо структуру датасету
data_yaml = os.path.join(YOLO_DATASET_DIR, "data.yaml")
if os.path.exists(data_yaml):
    print(f"Файл конфігурації створено: {data_yaml}")
    with open(data_yaml, 'r') as f:
        print("\nВміст data.yaml:")
        print(f.read())
    
    # Показуємо статистику
    for split in ['train', 'val', 'test']:
        images_dir = os.path.join(YOLO_DATASET_DIR, split, 'images')
        labels_dir = os.path.join(YOLO_DATASET_DIR, split, 'labels')
        if os.path.exists(images_dir):
            num_images = len([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
            num_labels = len([f for f in os.listdir(labels_dir) if f.endswith('.txt')])
            print(f"\n{split.upper()}: {num_images} зображень, {num_labels} labels")
else:
    print(f"ПОМИЛКА: Файл конфігурації не знайдено: {data_yaml}")


## 7. Навчання моделі YOLOv8


In [ ]:
# Параметри навчання
MODEL_SIZE = 'n'  # n, s, m, l, x (nano, small, medium, large, xlarge)
EPOCHS = 1
IMG_SIZE = 640
BATCH_SIZE = 16
DEVICE = 'cuda' if os.environ.get('CUDA_VISIBLE_DEVICES') else 'cpu'
PROJECT_NAME = 'artaxor_training'

print(f"Параметри навчання:")
print(f"   - Модель: YOLOv8{MODEL_SIZE}")
print(f"   - Епохи: {EPOCHS}")
print(f"   - Розмір зображення: {IMG_SIZE}")
print(f"   - Батч: {BATCH_SIZE}")
print(f"   - Пристрій: {DEVICE}")
print(f"   - Датасет: {data_yaml}")


In [ ]:
# Завантажуємо модель
model_name = f'yolov8{MODEL_SIZE}.pt'
print(f"Завантаження попередньо навченої моделі: {model_name}")
model = YOLO(model_name)

print("Модель завантажено успішно!")


In [ ]:
# Початок навчання
print("\n" + "="*60)
print("ПОЧАТОК НАВЧАННЯ")
print("="*60)

start_time = time.time()

results = model.train(
    data=data_yaml,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    project=OUTPUT_DIR,
    name=PROJECT_NAME,
    save=True,
    save_period=10,  # Зберігати чекпоінт кожні 10 епох
    val=True,  # Валідація під час навчання
    plots=True,  # Генерувати графіки
    verbose=True,  # Показувати детальний прогрес навчання
    seed=42,  # Для відтворюваності
    deterministic=True,
    amp=True,  # Automatic Mixed Precision (швидше навчання)
    cos_lr=True,  # Cosine learning rate schedule
    close_mosaic=10,  # Вимкнути mosaic за 10 епох до кінця
)

end_time = time.time()
training_time = end_time - start_time
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("\n" + "="*60)
print("НАВЧАННЯ ЗАВЕРШЕНО!")
print("="*60)
print(f"Час навчання: {hours:02d}:{minutes:02d}:{seconds:02d}")
print("="*60)


## 8. Результати навчання


In [ ]:
# Шляхи до результатів
# Спочатку намагаємося отримати шлях з об'єкта results
try:
    if 'results' in globals() and hasattr(results, 'save_dir'):
        results_dir = str(results.save_dir)
    elif 'results' in globals() and hasattr(results, 'save_path'):
        # Іноді YOLO зберігає шлях в save_path
        results_dir = os.path.dirname(str(results.save_path)) if results.save_path else os.path.join(OUTPUT_DIR, PROJECT_NAME)
    else:
        results_dir = os.path.join(OUTPUT_DIR, PROJECT_NAME)
except:
    results_dir = os.path.join(OUTPUT_DIR, PROJECT_NAME)

# Шукаємо модель в різних можливих місцях
best_model = None
last_model = None

# Можливі шляхи до моделей
possible_best_paths = [
    os.path.join(results_dir, 'weights', 'best.pt'),
    os.path.join(results_dir, 'best.pt'),
    os.path.join(OUTPUT_DIR, PROJECT_NAME, 'weights', 'best.pt'),
    os.path.join(OUTPUT_DIR, PROJECT_NAME, 'best.pt'),
]

possible_last_paths = [
    os.path.join(results_dir, 'weights', 'last.pt'),
    os.path.join(results_dir, 'last.pt'),
    os.path.join(OUTPUT_DIR, PROJECT_NAME, 'weights', 'last.pt'),
    os.path.join(OUTPUT_DIR, PROJECT_NAME, 'last.pt'),
]

# Також перевіряємо, чи є weights директорія і шукаємо там
weights_dir = os.path.join(results_dir, 'weights')
if os.path.exists(weights_dir):
    # Шукаємо всі .pt файли в weights
    for file in os.listdir(weights_dir):
        if file.endswith('.pt'):
            file_path = os.path.join(weights_dir, file)
            if 'best' in file.lower() and best_model is None:
                best_model = file_path
            elif 'last' in file.lower() and last_model is None:
                last_model = file_path

# Якщо не знайшли через список файлів, перевіряємо стандартні шляхи
if best_model is None:
    for path in possible_best_paths:
        if os.path.exists(path):
            best_model = path
            break

if last_model is None:
    for path in possible_last_paths:
        if os.path.exists(path):
            last_model = path
            break

print(f"Результати збережено в: {results_dir}")
if best_model:
    print(f"Найкраща модель: {best_model}")
else:
    print("Найкраща модель не знайдена")
    # Показуємо структуру директорії для діагностики
    if os.path.exists(results_dir):
        print(f"\nСтруктура директорії {results_dir}:")
        for root, dirs, files in os.walk(results_dir):
            level = root.replace(results_dir, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... та ще {len(files) - 5} файлів")

if last_model:
    print(f"Остання модель: {last_model}")
else:
    print("Остання модель не знайдена")

# Виводимо метрики
if hasattr(results, 'results_dict'):
    print("\nМетрики навчання:")
    for key, value in results.results_dict.items():
        print(f"   {key}: {value}")

# Показуємо графіки результатів
results_png = os.path.join(results_dir, 'results.png')
if os.path.exists(results_png):
    print(f"\nГрафіки результатів: {results_png}")
    from IPython.display import Image, display
    display(Image(results_png))


## 9. Валідація моделі (опціонально)


In [ ]:
# Валідація найкращої моделі
if best_model and os.path.exists(best_model):
    print(f"Валідація моделі: {best_model}")
    validation_model = YOLO(best_model)
    
    val_results = validation_model.val(
        data=data_yaml,
        imgsz=IMG_SIZE,
        device=DEVICE
    )
    
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТИ ВАЛІДАЦІЇ:")
    print("="*60)
    print(f"   mAP50: {val_results.box.map50:.4f}")
    print(f"   mAP50-95: {val_results.box.map:.4f}")
    if hasattr(val_results.box, 'mp'):
        print(f"   Precision: {val_results.box.mp:.4f}")
    if hasattr(val_results.box, 'mr'):
        print(f"   Recall: {val_results.box.mr:.4f}")
    print("="*60)
else:
    if best_model:
        print(f"ПОМИЛКА: Модель не знайдено: {best_model}")
    else:
        print("ПОМИЛКА: Модель не знайдено. Перевірте, чи навчання завершилося успішно.")


## 10. Тестування на прикладі зображення (опціонально)


In [ ]:
# Знаходимо тестове зображення
test_images_dir = os.path.join(YOLO_DATASET_DIR, 'test', 'images')
if os.path.exists(test_images_dir):
    test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if test_images:
        test_image_path = os.path.join(test_images_dir, test_images[0])
        print(f"Тестування на зображенні: {test_image_path}")
        
        if best_model and os.path.exists(best_model):
            test_model = YOLO(best_model)
            
            # Передбачення
            predictions = test_model.predict(
                source=test_image_path,
                conf=0.25,
                save=True,
                save_dir=os.path.join(OUTPUT_DIR, 'predictions'),
                show_labels=True,
                show_boxes=True
            )
            
            # Показуємо результат
            for result in predictions:
                print(f"\nЗнайдено об'єктів: {len(result.boxes)}")
                if len(result.boxes) > 0:
                    print("\nДеталі детекцій:")
                    for i, box in enumerate(result.boxes):
                        cls = int(box.cls[0])
                        conf = float(box.conf[0])
                        class_name = test_model.names[cls]
                        print(f"   {i+1}. {class_name}: {conf:.2%}")
                    
                    # Показуємо зображення з детекціями
                    from IPython.display import Image, display
                    result_path = os.path.join(OUTPUT_DIR, 'predictions', test_images[0])
                    if os.path.exists(result_path):
                        display(Image(result_path))
        else:
            print(f"ПОМИЛКА: Модель не знайдено: {best_model}")
    else:
        print(f"ПОМИЛКА: Тестові зображення не знайдено в {test_images_dir}")
else:
    print(f"ПОМИЛКА: Тестова директорія не знайдено: {test_images_dir}")


## Примітки

- Для кращих результатів рекомендується використовувати GPU
- Розмір батчу залежить від доступної пам'яті GPU
- Моделі більшого розміру (l, x) потребують більше пам'яті та часу навчання
- Рекомендується почати з моделі `n` (nano) для швидкого тестування
- Змініть параметри `MODEL_SIZE`, `EPOCHS`, `BATCH_SIZE` у комірці 7 для налаштування навчання
